In [1]:
import requests
from urllib.parse import urlencode

In [2]:
standings_url="https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
headers = {
    'authority': 'www.google.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    # Add more headers as needed
}

data=requests.get(standings_url, headers=headers)

In [59]:
import re
datastr=str(data)
x=re.findall("[0-9][0-9][0-9]",datastr)
x[0]

'200'

In [5]:
from bs4 import BeautifulSoup

In [6]:
soup=BeautifulSoup(data.text)

In [7]:
standings_table=soup.select('table.stats_table')[0]

In [8]:
links=standings_table.find_all('a')

In [9]:
links=[l.get("href") for l in links]

In [10]:
links=[l for l in links if "/squads/" in l]
links

['/en/squads/822bd0ba/Liverpool-Stats',
 '/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/squads/18bb7c10/Arsenal-Stats',
 '/en/squads/8602292d/Aston-Villa-Stats',
 '/en/squads/361ca564/Tottenham-Hotspur-Stats',
 '/en/squads/19538871/Manchester-United-Stats',
 '/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 '/en/squads/7c21e445/West-Ham-United-Stats',
 '/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 '/en/squads/b2b47a98/Newcastle-United-Stats',
 '/en/squads/cff3d9bb/Chelsea-Stats',
 '/en/squads/fd962109/Fulham-Stats',
 '/en/squads/47c64c55/Crystal-Palace-Stats',
 '/en/squads/4ba7cbea/Bournemouth-Stats',
 '/en/squads/d3fd31cc/Everton-Stats',
 '/en/squads/cd051869/Brentford-Stats',
 '/en/squads/e4a775cb/Nottingham-Forest-Stats',
 '/en/squads/e297cd13/Luton-Town-Stats',
 '/en/squads/943e8050/Burnley-Stats',
 '/en/squads/1df6b87e/Sheffield-United-Stats']

In [11]:
team_urls=[f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/s

In [102]:
import pandas as pd
from datetime import date
from io import StringIO
from datetime import datetime, timedelta

In [105]:
today = pd.to_datetime(datetime.now())
today

Timestamp('2024-03-02 15:59:32.163669')

In [119]:
end_date=pd.to_datetime(datetime.today()+timedelta(days=30))
end_date

Timestamp('2024-04-01 16:19:54.682255')

In [111]:
schmatches=[]

In [113]:
for team_url in team_urls:
    data=requests.get(team_url)
    matches=pd.read_html(StringIO(data.text), match="Scores & Fixtures")
    scheduled=matches[0][matches[0]["Comp"]=="Premier League"]
    scheduled["Formation"]=scheduled["Formation"].ffill(axis=0)
    scheduled["Poss"]=scheduled["Poss"].ffill(axis=0)
    scheduled["Date"]=pd.to_datetime(scheduled["Date"])
    scheduled=scheduled[scheduled["Date"]>=today][scheduled["Date"]<=end_date]
    scheduled.columns=[c.lower() for c in scheduled.columns]
    scheduled.iloc[:,:-1]=scheduled.iloc[:,:-1].ffill(axis = 0)
    schmatches.append(scheduled)

C:\Users\hp\AppData\Local\Temp\ipykernel_10384\1627709392.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scheduled["Formation"]=scheduled["Formation"].ffill(axis=0)
C:\Users\hp\AppData\Local\Temp\ipykernel_10384\1627709392.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scheduled["Poss"]=scheduled["Poss"].ffill(axis=0)
C:\Users\hp\AppData\Local\Temp\ipykernel_10384\1627709392.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [114]:
scheduled_matches=pd.concat(schmatches)

In [115]:
scheduled_matches=scheduled_matches.drop(["gf","ga","xg","xga","attendance","captain","referee","match report","notes"],axis=1)

In [116]:
scheduled_matches.tail()

,date,time,comp,round,day,venue,result,opponent,poss,formation
33,2024-03-30,15:00,Premier League,Matchweek 30,Sat,Away,NaN,Chelsea,44.0,4-4-2
29,2024-03-04,20:00,Premier League,Matchweek 27,Mon,Home,NaN,Arsenal,37.0,3-5-2
30,2024-03-09,15:00,Premier League,Matchweek 28,Sat,Away,NaN,Bournemouth,37.0,3-5-2
31,2024-03-16,15:00,Premier League,Matchweek 29,Sat,Away,NaN,Manchester Utd,37.0,3-5-2
32,2024-03-30,15:00,Premier League,Matchweek 30,Sat,Home,NaN,Fulham,37.0,3-5-2


In [117]:
scheduled_matches.index=[i for i in range(0,len(scheduled_matches.index))]
scheduled_matches.index

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67],
      dtype='int64')

In [118]:
scheduled_matches.to_json(index=False,date_format="iso",orient='records')

'[{"date":"2024-03-10T00:00:00.000","time":"15:45","comp":"Premier League","round":"Matchweek 28","day":"Sun","venue":"Home","result":null,"opponent":"Manchester City","poss":60.0,"formation":"4-3-3"},{"date":"2024-03-17T00:00:00.000","time":"14:00","comp":"Premier League","round":"Matchweek 29","day":"Sun","venue":"Away","result":null,"opponent":"Everton","poss":60.0,"formation":"4-3-3"},{"date":"2024-03-31T00:00:00.000","time":"14:00","comp":"Premier League","round":"Matchweek 30","day":"Sun","venue":"Home","result":null,"opponent":"Brighton","poss":60.0,"formation":"4-3-3"},{"date":"2024-03-03T00:00:00.000","time":"15:30","comp":"Premier League","round":"Matchweek 27","day":"Sun","venue":"Home","result":null,"opponent":"Manchester Utd","poss":62.0,"formation":"3-2-4-1"},{"date":"2024-03-10T00:00:00.000","time":"15:45","comp":"Premier League","round":"Matchweek 28","day":"Sun","venue":"Away","result":null,"opponent":"Liverpool","poss":62.0,"formation":"3-2-4-1"},{"date":"2024-03-17T0